In [1]:
#ignore warnings for clear printing
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
import seaborn as sns # More snazzy plotting library
import itertools

file_path = "/home/user/projects/data_mining/predictive_data_mining/logistic_regression/working_dataset.csv"
dta = pd.read_csv(file_path)
#clean up data non numeric rows
str_list = [] # empty list to contain columns with strings (words)
for colname, colvalue in dta.iteritems():
    if type(colvalue[1]) == str:
        #if colname not in str_list:
            str_list.append(colname)
# Get to the numeric columns by inversion
num_list = dta.columns.difference(str_list)
#USe only the numeriv values
dta_clean = dta[num_list]
#remove the null values, that is fill NaN with there - FIXME: Rihards, naive implementation
#dta_clean = dta_clean.fillna(value=0, axis=1)
dta_clean = dta_clean.dropna()
dta_clean = dta_clean.reindex_axis(sorted(dta_clean.columns), axis=1)
#clean up data from zero rows 
for colname, colvalue in dta_clean.iteritems():
    if colname != 'facenumber_in_poster':
        dta_clean = dta_clean[dta_clean[colname] != 0]
dta_clean.count()
#clasify the data for the logistic regression
def label_gross (gross):
    if (gross < 1000000) : return 1
    elif ((gross >= 1000000) & (gross < 10000000)) : return 2
    elif ((gross >= 10000000) & (gross < 50000000)) : return 3
    elif ((gross >= 50000000) & (gross < 200000000)) : return 4
    elif (gross >= 200000000) : return 5

y_target = dta_clean.gross.apply (lambda gross: label_gross (gross))
y_target = np.ravel(y_target)
x_data = dta_clean.drop('gross', axis=1)
#x_data = x_data.drop('aspect_ratio', axis=1)
#x_data = x_data.drop('duration', axis=1)
#x_data = x_data.drop('director_facebook_likes', axis=1)
dta_clean['gross_class'] = dta_clean.gross.apply (lambda gross: label_gross (gross))

#standarlisze
for colname, colvalue in x_data.iteritems():
                standard_scaler = preprocessing.StandardScaler().fit(x_data[colname])
                x_data[colname] = standard_scaler.transform(x_data[colname])


#calculate number of different componets in polynomal transfor
#poly_n_output_features = []
#for deg in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]:
#    poly = PolynomialFeatures(deg).fit(x_data)
#    poly_n_output_features.append(poly.n_output_features_)
#print poly_n_output_features
poly_n_output_features = [16, 136, 816, 3876, 15504, 54264, 170544, 490314, 1307504, 3268760, 7726160, 17383860, 37442160, 77558760, 155117520, 300540195]

def run_grid_search_pca(x,y,name, degrees, n_components, solvers, class_weight, Cs):
    #create pipline and use GridSearch to find the betst params
    polynomial = PolynomialFeatures().fit(x_data)
    pca = decomposition.PCA().fit(x_data) 
    logistic = linear_model.LogisticRegression()
    pipe = Pipeline(steps=[('polynomial', polynomial),('pca', pca), ('logistic', logistic)])
    #create estimator
    estimator = GridSearchCV(pipe,
                             dict(polynomial__degree = degrees,
                                  pca__n_components = n_components,
                                  logistic__C = Cs,                           
                                  logistic__solver = solvers,
                                  logistic__class_weight = class_weight)
                             ,verbose=2, cv=5, n_jobs=4)
    #run the esmimator    
    estimator.fit(x, y)
    print "GREP_ME***Results of ["  + name + "] estimator run are"
    print estimator.cv_results_
    print "GREP_ME***Best params of ["  + name + "] estimator run are"
    print estimator.best_params_
    print "GREP_ME***Best score of ["  + name + "] estimator run are"
    print estimator.best_score_
    
def run_grid_search_fa(x,y,name, degrees, n_components, solvers, class_weight, Cs):
    #create pipline and use GridSearch to find the betst params
    polynomial = PolynomialFeatures().fit(x_data)
    fa = decomposition.FactorAnalysis().fit(x_data) 
    logistic = linear_model.LogisticRegression()
    pipe = Pipeline(steps=[('polynomial', polynomial),('fa', fa), ('logistic', logistic)])
    #create estimator
    estimator = GridSearchCV(pipe,
                             dict(polynomial__degree = degrees,
                                  fa__n_components = n_components,
                                  logistic__C = Cs,                           
                                  logistic__solver = solvers,
                                  logistic__class_weight = class_weight)
                             ,verbose=2, cv=5, n_jobs=4)
    #run the esmimator    
    estimator.fit(x, y)
    print "GREP_ME***Results of ["  + name + "] estimator run are"
    print estimator.cv_results_
    print "GREP_ME***Best params of ["  + name + "] estimator run are"
    print estimator.best_params_
    print "GREP_ME***Best score of ["  + name + "] estimator run are"
    print estimator.best_score_

def run():
    ####################################################
    # Process a simple first degree pipeline with PCA  #
    ###################################################
    degrees=[1]
    n_components = [4, 5, 6, 7, 8, 9, 10, 11, 12, 15]
    solvers =  ['newton-cg', 'lbfgs', 'liblinear', 'sag']
    class_weight = [None, "balanced"]
    Cs = np.logspace(-4, 4, 3)
    run_grid_search_pca(x_data, y_target, "First degree, PCA - no mle", degrees, n_components, solvers, class_weight, Cs)

    ###############################################
    # Process a 5 degree pipeline with  PCA mle  #
    ###############################################
    degrees=[1,2,3,4,5]
    n_components = ['mle']
    solvers =  ['newton-cg', 'lbfgs', 'liblinear', 'sag']
    class_weight = [None, "balanced"]
    Cs = np.logspace(-4, 4, 3)
    run_grid_search_pca(x_data, y_target, "5 degrees, PCA with mle", degrees, n_components, solvers, class_weight, Cs)

    ##########################################
    # Process a 5 degree pipeline with FA   #
    ##########################################
    for deg in [1, 2,3,4,5]:
        degrees=[deg]
        n_components = [4, 5, 6, 7, 8, 9, 10, 11, 12, poly_n_output_features[deg-1] - 1]
        solvers =  ['newton-cg', 'lbfgs', 'liblinear', 'sag']
        class_weight = [None, "balanced"]
        Cs = np.logspace(-4, 4, 3)
        print n_components
        string = str(deg) + " degree, FA"
        run_grid_search_fa(x_data, y_target, string, degrees, n_components, solvers, class_weight, Cs)

#ignore warnigs
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    run()


Fitting 5 folds for each of 240 candidates, totalling 1200 fits


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either usi

[CV] logistic__C=0.0001, logistic__solver=newton-cg, pca__n_components=4, logistic__class_weight=None, polynomial__degree=1 
[CV] logistic__C=0.0001, logistic__solver=newton-cg, pca__n_components=4, logistic__class_weight=None, polynomial__degree=1 
[CV] logistic__C=0.0001, logistic__solver=newton-cg, pca__n_components=4, logistic__class_weight=None, polynomial__degree=1 
[CV] logistic__C=0.0001, logistic__solver=newton-cg, pca__n_components=4, logistic__class_weight=None, polynomial__degree=1 
[CV]  logistic__C=0.0001, logistic__solver=newton-cg, pca__n_components=4, logistic__class_weight=None, polynomial__degree=1, total=   0.1s
[CV] logistic__C=0.0001, logistic__solver=newton-cg, pca__n_components=4, logistic__class_weight=None, polynomial__degree=1 
[CV]  logistic__C=0.0001, logistic__solver=newton-cg, pca__n_components=4, logistic__class_weight=None, polynomial__degree=1, total=   0.2s
[CV]  logistic__C=0.0001, logistic__solver=newton-cg, pca__n_components=4, logistic__class_weig

[Parallel(n_jobs=4)]: Done  58 tasks      | elapsed:    2.5s


[CV]  logistic__C=0.0001, logistic__solver=lbfgs, pca__n_components=7, logistic__class_weight=None, polynomial__degree=1, total=   0.1s
[CV] logistic__C=0.0001, logistic__solver=lbfgs, pca__n_components=7, logistic__class_weight=None, polynomial__degree=1 
[CV] logistic__C=0.0001, logistic__solver=lbfgs, pca__n_components=7, logistic__class_weight=None, polynomial__degree=1 
[CV]  logistic__C=0.0001, logistic__solver=lbfgs, pca__n_components=7, logistic__class_weight=None, polynomial__degree=1, total=   0.1s
[CV]  logistic__C=0.0001, logistic__solver=lbfgs, pca__n_components=8, logistic__class_weight=None, polynomial__degree=1, total=   0.1s
[CV] logistic__C=0.0001, logistic__solver=lbfgs, pca__n_components=8, logistic__class_weight=None, polynomial__degree=1 
[CV] logistic__C=0.0001, logistic__solver=lbfgs, pca__n_components=8, logistic__class_weight=None, polynomial__degree=1 
[CV]  logistic__C=0.0001, logistic__solver=lbfgs, pca__n_components=7, logistic__class_weight=None, polynomi

...

[Parallel(n_jobs=4)]: Done 706 tasks      | elapsed:   27.3s


[CV]  logistic__C=1.0, logistic__solver=liblinear, pca__n_components=6, logistic__class_weight=balanced, polynomial__degree=1, total=   0.1s
[CV] logistic__C=1.0, logistic__solver=liblinear, pca__n_components=8, logistic__class_weight=balanced, polynomial__degree=1 
[CV] logistic__C=1.0, logistic__solver=liblinear, pca__n_components=8, logistic__class_weight=balanced, polynomial__degree=1 
[CV] logistic__C=1.0, logistic__solver=liblinear, pca__n_components=8, logistic__class_weight=balanced, polynomial__degree=1 
[CV]  logistic__C=1.0, logistic__solver=liblinear, pca__n_components=7, logistic__class_weight=balanced, polynomial__degree=1, total=   0.0s
[CV] logistic__C=1.0, logistic__solver=liblinear, pca__n_components=9, logistic__class_weight=balanced, polynomial__degree=1 
[CV]  logistic__C=1.0, logistic__solver=liblinear, pca__n_components=8, logistic__class_weight=balanced, polynomial__degree=1, total=   0.1s
[CV]  logistic__C=1.0, logistic__solver=liblinear, pca__n_components=8, l

[Parallel(n_jobs=4)]: Done 1200 out of 1200 | elapsed:   51.0s finished


GREP_ME***Results of [First degree, PCA - no mle] estimator run are
{'std_train_score': array([ 0.00061182,  0.00113448,  0.00077868,  0.00112293,  0.00092459,
        0.00117377,  0.00102966,  0.00102966,  0.00102966,  0.00117377,
        0.00061182,  0.00113448,  0.00077868,  0.00112293,  0.00092459,
        0.00117377,  0.00102966,  0.00102966,  0.00102966,  0.00117377,
        0.00299677,  0.00414534,  0.00478786,  0.00491132,  0.00597836,
        0.00637992,  0.00495188,  0.00483722,  0.00553251,  0.00651334,
        0.00061182,  0.00085916,  0.00069601,  0.00112293,  0.00116126,
        0.00117377,  0.00102966,  0.00116126,  0.0009923 ,  0.00117377,
        0.01100118,  0.01222535,  0.0145029 ,  0.008397  ,  0.01628517,
        0.01360373,  0.01127118,  0.00920322,  0.00891841,  0.01021912,
        0.01081265,  0.01238302,  0.0145029 ,  0.008397  ,  0.01628517,
        0.01360373,  0.01111166,  0.00929937,  0.00891841,  0.01035047,
        0.01013398,  0.0049951 ,  0.0089226 ,  0

KeyboardInterrupt: 

plt.plot(np.cumsum(pca.explained_variance_ratio_), linewidth=2)
plt.xlabel('n_components')
plt.axis('tight')
plt.ylabel('explained_variance_')


In [ ]:
plt.axvline(estimator.best_estimator_.named_steps['pca'].n_components, linestyle=':', label='n_components chosen')
plt.legend(prop=dict(size=12))
plt.show()

In [ ]:
estimator.best_params_

In [ ]:
estimator.best_estimator_

In [ ]:
#

In [ ]:
pcaa = decomposition.PCA(n_components='mle')
x_9d = pcaa.fit_transform(x_data)
pd.DataFrame(x_9d)

In [ ]:
pd.DataFrame(x_data)

In [ ]:
pd.DataFrame(pcaa.components_,columns=x_data.columns)

In [ ]:
pcaa = decomposition.PCA(n_components=13)
x_9d = pcaa.fit_transform(x_data)
pd.DataFrame(x_9d)

In [ ]:
for colname, colvalue in dta_clean.iteritems():
    if colname != 'gross' and colname != 'gross_class':
                standard_scaler = preprocessing.StandardScaler().fit(dta_clean[colname])
                dta_clean[colname] = standard_scaler.transform(dta_clean[colname])

for colname, colvalue in dta_clean.iteritems():
    if colname != 'gross' and colname != 'gross_class':
        g = sns.lmplot(x=colname, y="gross", data=dta_clean, order=1)  


In [ ]:
for colname, colvalue in dta_clean.iteritems():
    if colname != 'gross' and colname != 'gross_class':
                standard_scaler = preprocessing.StandardScaler().fit(dta_clean[colname])
                dta_clean[colname] = standard_scaler.transform(dta_clean[colname])

for colname, colvalue in dta_clean.iteritems():
    if colname != 'gross' and colname != 'gross_class':
                poly = PolynomialFeatures(2).fit(dta_clean[colname])
                dta_clean[colname] = poly.transform(dta_clean[colname])              

for colname, colvalue in dta_clean.iteritems():
    if colname != 'gross' and colname != 'gross_class':
        g = sns.lmplot(x=colname, y="gross", data=dta_clean, order=2)  

In [ ]:
poly = PolynomialFeatures(2)
poly_fit = poly.fit_transform(dta_clean) 

In [ ]:
target_feature_names = ['x'.join(['{}^{}'.format(pair[0],pair[1]) for pair in tuple if pair[1]!=0]) for tuple in [zip(dta_clean.columns,p) for p in poly.powers_]]
pd.DataFrame(poly_fit, columns = target_feature_names)

In [ ]:
poly

In [ ]:
dir(poly)

In [ ]:
poly.powers_

In [ ]:
poly.n_output_features_